# Import packages

In [2]:
import os
import pandas as pd
import json
# -*- coding:utf-8 -*-

## 将标注文档转成excel

In [18]:
input_path = "input"
sped_l = os.listdir(input_path)
ent = ["是否是致病菌", "样本量", "患者性别", "年龄", "免疫状态", "肺部基础疾病", "证据等级", "预后"]

for sped in sped_l:

    collection = {}
    paper_l = os.listdir("%s/%s/outputs" % (input_path, sped))
    paper_l = [ele for ele in paper_l if "json" in ele]

    for paper in paper_l:

        pattern = {
            "物种名称": "",
            "是否是致病菌": "",
            "样本量": "",
            "患者性别": "",
            "年龄": "",
            "免疫状态": "",
            "肺部基础疾病": "",
            "证据等级": "",
            "预后": ""
        }

        with open("%s/%s/outputs/%s" % (input_path, sped, paper),
                  "r",
                  encoding='UTF-8') as f:
            ano = json.load(f)

        evi_list = []
        evis_list = []

        if ano["outputs"] != {}:
            for i in ano["outputs"]["annotation"]["A"]:
                if len(i) > 1:

                    # 保存所有信息到pattern
                    if i["name"] in ent:
                        pattern[i["name"]] = i["value"]

                    # 获取标注的是否是病原菌的信息
                    if i["name"] == "证据等级":
                        evi_list.append(i["value"])

            # j为原文本
            for j in ano["outputs"]["annotation"]["T"]:
                if j is not None:
                    if len(j) > 1:

                        # 获取标注的判断病原菌的文本
                        if j["name"] == "物种":
                            pattern["物种名称"] = j["value"]

                        # 获取标注的判断证据等级的文本
                        if j["name"] == "证据等级":

                            # 比较等级
                            evis_list.append(j["value"])
                            if len(evis_list) > 1:
                                evi_new = evi_list[len(evis_list)-1]
                                evi_old = evi_list[len(evi_list)-2]
                                if evi_new < evi_old:
                                    pattern["证据等级"] == j["value"]
                                    # 获取证据等级所在的整个句子
                                    new_start = ano["content"].rfind(
                                        ".", 0, j["start"])
                                    new_end = ano["content"].find(
                                        ".", j["end"], -1)
                                    evi_txt = ano["content"][new_start+1:new_end+1]
                                    pattern["answer"] = j["value"]
                                    pattern["old_start"] = j["start"]
                                    pattern["old_end"] = j["end"]
                                    pattern["sup_evi"] = evi_txt
                                    pattern["new_start"] = new_start + 1
                                    pattern["new_end"] = new_end + 1
                            else:
                                new_start = ano["content"].rfind(
                                    ".", 0, j["start"])
                                new_end = ano["content"].find(
                                    ".", j["end"], -1)
                                evi_txt = ano["content"][new_start+1:new_end+1]
                                pattern["answer"] = j["value"]
                                pattern["old_start"] = j["start"]
                                pattern["old_end"] = j["end"]
                                pattern["sup_evi"] = evi_txt
                                pattern["new_start"] = new_start + 1
                                pattern["new_end"] = new_end + 1

            pattern["id"] = 'https://pubmed.ncbi.nlm.nih.gov/%s/' % paper.replace(
                ".json", "")

            collection[paper.replace(".json", "")] = pattern

    df = pd.DataFrame(collection).T
    df.to_excel("文献标注/%s.xls" % (sped))


<ipython-input-18-4b1ff039fb97>:95: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("文献标注/%s.xls" % (sped))


# 添加支持证据

In [3]:
add_list = [ele for ele in os.listdir("文献标注-校对") if "_add" in ele]
ori_list = [ele for ele in os.listdir("文献标注") if "-" in ele]
for add in add_list:
    df = pd.read_excel("文献标注-校对/%s" % add, header=0, index_col=0)
    
    dfo_l = []
    for ori in ori_list:
        if ori.split("-")[0] in add:
            dfo = pd.read_excel("文献标注/%s" % ori, header=0, index_col=0)
            dfo_l.append(dfo)
    for idx in dfo.index:
        if pd.isna(dfo.loc[idx, "sup_evi"]) is True:
            dfo.loc[idx, "answer"] =  dfo.loc[idx, "answer"]
            dfo.loc[idx, "old_start"] = dfo_l[0].loc[idx, "old_start"]
            dfo.loc[idx, "old_end"] = dfo_l[0].loc[idx, "old_end"]
            dfo.loc[idx, "sup_evi"] = dfo_l[0].loc[idx, "sup_evi"]
            dfo.loc[idx, "new_start"] = dfo_l[0].loc[idx, "new_start"]
            dfo.loc[idx, "new_end"] = dfo_l[0].loc[idx, "new_end"]
    df["sup_evi"] = dfo["sup_evi"]
    df.to_excel("文献标注-校对/%s" % add.replace("add", "env"))

<ipython-input-3-f0b1b73660a5>:20: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("文献标注-校对/%s" % add.replace("add", "env"))
